# CECS 456 Deep Learning Project

## Imports

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np
import os
import matplotlib.pyplot as plt
from collections import Counter
import random
from google.colab import drive
drive.mount("/content/drive")
! kaggle datasets download -d alessiocorrado99/animals10

! unzip /content/animals10.zip


## Define constants

In [ ]:

imgSize = 128
batchSize = 32
animalsCnt = 10
epochs = 10
datasetPath = "raw-img" # Path to images

#Translation
translate = {"cane": "dog", "cavallo": "horse", "elefante": "elephant", "farfalla": "butterfly", "gallina": "chicken", "gatto": "cat", "mucca": "cow", "pecora": "sheep", "scoiattolo": "squirrel", "dog": "cane", "cavallo": "horse", "elephant" : "elefante", "butterfly": "farfalla", "chicken": "gallina", "cat": "gatto", "cow": "mucca", "spider": "ragno", "squirrel": "scoiattolo"}


## Data preparation & load dataset

In [ ]:

# Data augmentation
datagen = ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.15,
    horizontal_flip = True,
    zoom_range = 0.2,
)

# Load training data
train_data = datagen.flow_from_directory(
    datasetPath,
    target_size = (imgSize, imgSize),
    batch_size = batchSize,
    class_mode = 'categorical',
    subset = 'training',
    shuffle = True
)

# Load validation data
val_data = datagen.flow_from_directory(
    datasetPath,
    target_size = (imgSize, imgSize),
    batch_size = batchSize,
    class_mode = 'categorical',
    subset = 'validation',
    shuffle = True
)

# Visualize dataset distribution across classes

In [ ]:

# Display image distribution across dataset
classes = train_data.class_indices
class_counts = Counter(train_data.classes)

plt.figure(figsize=(10, 6))
plt.bar(classes.keys(), class_counts.values(), color='skyblue')
plt.xlabel('Animal Classes')
plt.ylabel('Number of Images')
plt.title('Dataset Distribution')
plt.xticks(rotation = 45)
plt.tight_layout()
plt.show()


## Examples from dataset

In [ ]:
# Display images from the dataset with class labels
plt.figure(figsize=(12, 12))
for i, (image, label) in enumerate(train_data):
    if i >= 9:
        break
    plt.subplot(3, 3, i + 1)
    plt.imshow(image[0])
    translatedLabel = [k for k, v in classes.items() if v == np.argmax(label[0])][0]
    plt.title(f"Class: {translate.get(translatedLabel, translatedLabel)}")
    plt.axis('off')
plt.tight_layout()
plt.show()


# Define, build, and compile the model for training

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam

# Define VGG Model
vggModel = VGG16(weights = 'imagenet', include_top = False, input_shape = (imgSize, imgSize, 3))
for layer in vggModel.layers[:-4]: # Freeze layers from vggModel
    layer.trainable = False


model = Sequential([
    vggModel,
    GlobalAveragePooling2D(),
    Dense(256, activation = 'relu'),
    Dropout(0.5),
    Dense(animalsCnt, activation = 'softmax')
])

model.compile(optimizer = Adam(learning_rate=0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

earlyStopping = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 3) # EarlyStopping after validation checks


history = model.fit(train_data, epochs = epochs, validation_data = val_data, callbacks = [earlyStopping]) # Training the model


## Evaluate accuracy and total loss on training data

In [ ]:

total_loss, total_accuracy = model.evaluate(train_data) # Evaluation of accuracy and loss
print("Accuracy on Training Set: ", total_accuracy)
print("Total loss on Training Set: ", total_loss)

## Plot training and validation accuracy/loss

In [ ]:
#Training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy')
plt.show()

#Training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()


## Predictions on validation images

In [ ]:

plt.figure(figsize=(12, 12)) # set size of figure
for i, (image, label) in enumerate(val_data):
    if i >= 9:
        break
    pred = model.predict(image) # get prediction
    predicted_class = [k for k, v in classes.items() if v == np.argmax(pred[0])][0] # get predicted classes
    actual_class = [k for k, v in classes.items() if v == np.argmax(label[0])][0] # get actual classes
    plt.subplot(3, 3, i + 1)
    plt.imshow(image[0])
    plt.title(f"Actual: {translate.get(actual_class, actual_class)}, Pred: {translate.get(predicted_class, predicted_class)}")
    plt.axis('off')
plt.tight_layout()
plt.show()